In [12]:
library(mgcv)
library(sf)
library(tidyverse)
library(gstat)

In [6]:
# load the data
pad_voronoi <- read_sf("pad_voronoi.shp")
germany <- pad_voronoi %>% 
  st_geometry() %>%
  st_union()

pad_mds <- read_csv("pad_mds.csv") %>%
  st_as_sf(coords = c("LONGITUDE", "LATITUDE")) # 183 obs in total
st_crs(pad_mds) <- 4326

pad_grid <- germany %>% 
  st_bbox() %>%
  st_as_sfc() %>%
  st_make_grid(
    cellsize = c(.05, .05),
    what="centers"
  ) %>% 
  st_as_sf(crs=4326)
st_crs(pad_mds) <- 4326

Rows: 183 Columns: 7
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): doculect, col
dbl (5): LONGITUDE, LATITUDE, r, g, b

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [8]:
# divide into training and test set
pad_train = sample_n(pad_mds, 165) # 90% of the data for training
st_crs(pad_train) <- 4326

outside <- sapply(st_intersects(pad_mds, pad_train),function(x){length(x)==0})
pad_test = pad_mds[outside, ] # 10% of the data for testing
st_crs(pad_test) <- 4326

In [15]:
# Inverse distance weighted interpolation
pad.idw <- idw(r ~ 1, location = pad_train, newdata = pad_test, idp=2)
pad.idw

[inverse distance weighted interpolation]


Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson



,var1.pred,var1.var,geometry
,<dbl>,<dbl>,<POINT [°]>
1,0.7687961,NA,POINT (11.81097 48.19687)
2,0.7266071,NA,POINT (11.07393 49.45431)
3,0.7147772,NA,POINT (12.47554 48.09406)
4,0.6814563,NA,POINT (9.81708 49.89864)
5,0.5715453,NA,POINT (8.83433 50.21787)
6,0.6622855,NA,POINT (7.63783 49.60773)
7,0.5750662,NA,POINT (6.78431 49.95674)
8,0.4842068,NA,POINT (7.78567 50.57188)
9,0.3719699,NA,POINT (9.93846 51.25775)


In [16]:
# Ordinary Kriging
pad.v <- pad_train %>% variogram(r ~ 1, ., cloud=F, cutoff=1000) 
myVariogramModel <- vgm(psill=0.28, "Sph", range=1000, nugget=0.02)
pad.vfit <- fit.variogram(pad.v, myVariogramModel, fit.ranges=F)
pad.krige <- krige(r ~ 1, pad_train, pad_test, pad.vfit)
pad.krige

[using ordinary kriging]


,var1.pred,var1.var,geometry
,<dbl>,<dbl>,<POINT [°]>
1,0.8464249,0.013401733,POINT (11.81097 48.19687)
2,0.8413702,0.009807277,POINT (11.07393 49.45431)
3,0.7632074,0.013701823,POINT (12.47554 48.09406)
4,0.7994210,0.007752890,POINT (9.81708 49.89864)
5,0.6195221,0.010325068,POINT (8.83433 50.21787)
6,0.7416569,0.010045375,POINT (7.63783 49.60773)
7,0.6607217,0.012211358,POINT (6.78431 49.95674)
8,0.3705807,0.008226260,POINT (7.78567 50.57188)
9,0.2525663,0.006991102,POINT (9.93846 51.25775)


In [20]:
# Generalized Additive Models
gam.fit <- pad_train %>%
  cbind(., st_coordinates(.)) %>%
  select(r, X, Y) %>%
  mgcv::gam(r ~ s(X,Y), data=.) 

gam.prediction <- predict(gam.fit, newdata = data.frame(st_coordinates(pad_test)))

pad.gam <- pad.krige %>%
  cbind(., st_coordinates(.)) %>%
  st_drop_geometry() %>%
  mutate(Z = gam.prediction) %>%
  select(X, Y, Z) 
#   %>% raster::rasterFromXYZ(crs=4326) %>%
#   as("SpatRaster")

pad.gam

,X,Y,Z
,<dbl>,<dbl>,<dbl>
1,11.81097,48.19687,0.8324694
2,11.07393,49.45431,0.8354578
3,12.47554,48.09406,0.7793219
4,9.81708,49.89864,0.7754256
5,8.83433,50.21787,0.6719078
6,7.63783,49.60773,0.7321496
7,6.78431,49.95674,0.6638333
8,7.78567,50.57188,0.5525381
9,9.93846,51.25775,0.3791023
